<a href="https://colab.research.google.com/github/rapha18th/Kaggle/blob/main/sozograph_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SozoGraph Notebook Example


### Install package

In [21]:
!pip -q install -U sozograph

import os
from datetime import datetime, timezone

from sozograph import SozoGraph, Passport

### Setup Gemini

In [16]:
# Option A: paste key directly (fast for demo)
os.environ["GEMINI_API_KEY"] = os.environ.get("GEMINI_API_KEY", "***********************")

# Option B: if already set in environment, do nothing
API_KEY = os.environ.get("GEMINI_API_KEY") or os.environ.get("GOOGLE_API_KEY")
assert API_KEY and "PASTE_" not in API_KEY, "Set GEMINI_API_KEY first"

In [22]:
sg = SozoGraph(api_key=API_KEY, extractor_model="gemini-3-flash-preview")
#setup cognitive passport
passport = Passport()

### Example transcript

In [25]:
conversation = [
    # fact
    "I live in Harare.",
    # preference
    "I prefer direct communication. Don't sugar-coat responses.",
    # fact update (temporal truth)
    "I moved to Kwekwe last month.",
    # new stable fact
    "I'm building SozoGraph as a portable cognitive passport for agents.",
]

### Ingest data

In [26]:
passport, stats_list = sg.ingest(conversation, passport=passport)

print("STATS (per interaction):")
for i, s in enumerate(stats_list, start=1):
    print(f"{i}. {s}")

STATS (per interaction):
1. ResolveStats(facts_upserted=1, prefs_upserted=0, entities_merged=1, open_loops_added=0, contradictions_added=0)
2. ResolveStats(facts_upserted=0, prefs_upserted=2, entities_merged=0, open_loops_added=0, contradictions_added=0)
3. ResolveStats(facts_upserted=2, prefs_upserted=0, entities_merged=1, open_loops_added=0, contradictions_added=1)
4. ResolveStats(facts_upserted=2, prefs_upserted=0, entities_merged=1, open_loops_added=0, contradictions_added=0)


### Extract facts, preferences, contradictions

In [27]:
def show_facts(p):
    print("\nFACTS:")
    for f in p.facts:
        print(f"- {f.key} = {f.value} (source={f.source})")

def show_prefs(p):
    print("\nPREFERENCES:")
    for pr in p.prefs:
        print(f"- {pr.key} = {pr.value} (source={pr.source})")

def show_contradictions(p):
    print("\nCONTRADICTIONS:")
    if not p.contradictions:
        print("- (none)")
        return
    for c in p.contradictions:
        print(f"- {c.key}: {c.old} -> {c.new} (ts_old={c.ts_old.date()}, ts_new={c.ts_new.date()})")

show_facts(passport)
show_prefs(passport)
show_contradictions(passport)


FACTS:
- current_project = SozoGraph (source=i_3)
- location = Kwekwe (source=i_2)
- relocation_date = 2026-01 (source=i_2)
- sozograph_description = portable cognitive passport for agents (source=i_3)

PREFERENCES:
- communication_style = direct (source=i_1)
- response_tone = no sugar-coating (source=i_1)

CONTRADICTIONS:
- location: Harare -> Kwekwe (ts_old=2026-02-04, ts_new=2026-02-04)


### SozoGraph passport, the memory object to pass to an Agent

In [28]:
context = sg.export_context(passport, budget_chars=1200)
print(context)

SOZOGRAPH PASSPORT v1
Updated: 2026-02-04T15:21:41.829685+00:00

Facts (current beliefs):
- current_project: SozoGraph
- sozograph_description: portable cognitive passport for agents
- location: Kwekwe
- relocation_date: 2026-01

Preferences:
- communication_style: direct
- response_tone: no sugar-coating

Key entities:
- Harare (place)
- Kwekwe (place)
- SozoGraph (project)

Recent updates (contradictions resolved by time):
- location changed: Harare -> Kwekwe


### ResolveStats show what changed per interaction.

In [29]:
passport, stats_list = sg.ingest(conversation, passport=passport)

print("STATS (per interaction):")
for i, s in enumerate(stats_list, start=1):
    print(f"{i}. {s}")

STATS (per interaction):
1. ResolveStats(facts_upserted=1, prefs_upserted=0, entities_merged=1, open_loops_added=0, contradictions_added=1)
2. ResolveStats(facts_upserted=0, prefs_upserted=0, entities_merged=0, open_loops_added=0, contradictions_added=0)
3. ResolveStats(facts_upserted=1, prefs_upserted=0, entities_merged=1, open_loops_added=0, contradictions_added=0)
4. ResolveStats(facts_upserted=1, prefs_upserted=0, entities_merged=1, open_loops_added=0, contradictions_added=0)
